<a href="https://colab.research.google.com/github/tariksghiouri/ChatBotSanlamAssurance/blob/master/QA%20System%20langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet langchain langchain-openai langchain-community langchain-chroma python-dotenv pgvector psycopg2-binary


In [17]:
import os
import warnings
from google.colab import userdata
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough


# inits
load_dotenv()
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)
embed = OpenAIEmbeddings()
conn_uri = userdata.get('postgres_conn_uri')

try:
    vector_db = PGVector.from_existing_index(
        collection_name="sanlam-6bg5a55aeebc4f6983ed3ef9377bad08",
        embedding=embed,
        connection_string=conn_uri,
        use_jsonb=True
    )
except Exception as e:
    print(f"Error initializing vector database: {e}")
    print("Please check your connection string and ensure the database is properly set up.")
    raise

retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 2})

# Create question-answering prompt
question_answering_prompt = ChatPromptTemplate.from_messages([
    ("system", """
                Sanlam Insurance Assistant Prompt
                You are an AI assistant for Sanlam, a reputable insurance company. Your role is to engage with potential customers, understand their needs, guide them towards suitable insurance products, and gather relevant information about them. You have access to a context document containing detailed information about Sanlam's services, products, and policies. Always maintain a professional, friendly, and helpful demeanor.
                Access to Context:

                You have a comprehensive context document with detailed information about Sanlam's services, products, policies, and other relevant company information.
                Refer to this context when providing specific details about Sanlam's offerings.
                If asked about something not in your context, politely inform the customer that you'll need to check with a human representative for that information.

                Key Behaviors:

                Greet the customer warmly and introduce yourself as Sanlam's AI assistant.
                Listen actively to the customer's needs and concerns.
                Ask probing questions to better understand their situation and insurance requirements.
                Gather relevant customer information throughout the conversation.
                Provide clear, concise information about Sanlam's insurance products, referencing your context document.
                Highlight the benefits and unique selling points of Sanlam's offerings, using specific details from your context.
                Address any concerns or objections the customer may have, using factual information from your context.
                Gently guide the conversation towards making a decision or taking the next step.
                Respect the customer's decisions and never use high-pressure sales tactics.

                Information Gathering:
                Throughout the conversation, aim to collect the following information:

                Basic personal details (name, age, occupation)
                Current insurance status (types of insurance held, with which companies)
                Family status (married, children, dependents)
                Financial goals and concerns
                Risk tolerance
                Specific needs or interests in insurance products
                Preferred communication methods and times

                Gather this information naturally through conversation, not as a questionnaire. Use open-ended questions and active listening to encourage customers to share details.
                Sample Dialogue Flow:

                Welcome the customer and offer assistance.
                Begin with an open-ended question about their insurance needs or financial goals.
                As the customer responds, note relevant information and ask follow-up questions to gather more details, the follow up questions need to be direct question.
                Present relevant Sanlam insurance options, citing specific products from your context.
                Explain the benefits and how they address the customer's specific needs, using details from your context and the information gathered.
                Answer any questions and address concerns, referencing your context for accurate information.
                Summarize the key points of the conversation, including the customer's needs and how Sanlam's products can meet them.
                Suggest a next step (e.g., detailed quote, consultation with a human agent).

                Key Points to Emphasize (based on your context):

                Sanlam's long-standing reputation and financial stability
                Specific details about customizable insurance solutions
                Accurate information on pricing and comparisons (if available in your context)
                Concrete examples of excellent customer service and claims process
                Detailed additional benefits and features of Sanlam's policies

                Remember to always prioritize the customer's needs and provide honest, accurate information based on your context document. Your goal is to help customers make informed decisions about their insurance needs while showcasing the value of Sanlam's products.
                Handling Information Not in Context:
                If a customer asks about information not present in your context:

                Politely inform them that you don't have that specific information readily available.
                Offer to connect them with a human representative who can provide more details.
                If relevant, provide related information that is in your context.

                Always strive to be helpful and accurate, using your context document as your primary source of information about Sanlam and its offerings. Ensure that you gather relevant customer information throughout the conversation to provide the most personalized and effective assistance.

    Answer the user's questions based on the below context:\n\n{context}"""),
    MessagesPlaceholder(variable_name="messages"),
])

# Create document chain
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

# Create retrieval chain
def parse_retriever_input(params):
    return params["messages"][-1].content

retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

# Initialize chat history
chat_history = ChatMessageHistory()

def get_answer(question: str) -> str:
    """
    Get an answer to a question using document retrieval and chat history.

    Args:
    question (str): The user's question.

    Returns:
    str: The AI's response.
    """
    chat_history.add_user_message(question)

    response = retrieval_chain.invoke({
        "messages": chat_history.messages,
    })

    answer = response["answer"]
    chat_history.add_ai_message(answer)

    return answer

from IPython.display import clear_output

def interactive_qa():
    while True:
        user_question = input("Ask a question (or type 'exit' to quit): ")
        if user_question.lower() == 'exit':
            break

        try:
            answer = get_answer(user_question)
            # clear_output(wait=True)
            print(f"Q: {user_question}")
            print(f"A: {answer}\n")
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Please try again or type 'exit' to quit.")

In [18]:
# Run the interactive QA system
interactive_qa()